In [301]:
import pandas as pd
import numpy as np 

In [302]:
cafe_original = pd.read_csv(r'C:\Users\anws5\OneDrive\Documents\Python_projects\data_cleaning_project\dirty_cafe_sales.csv')
df = cafe_original.copy()


##  Initial Exploration

Before diving into cleaning or analysis, we'll take an initial look at the dataset to understand its  content, and potential issues.


In [303]:
df.head(5)

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [304]:
df.shape

(10000, 8)

In [305]:
df.columns = df.columns.str.replace(' ' , '_').str.capitalize()
df.columns

Index(['Transaction_id', 'Item', 'Quantity', 'Price_per_unit', 'Total_spent',
       'Payment_method', 'Location', 'Transaction_date'],
      dtype='object')

We can see there is some issue with the dataset but what I will do is clean the dataset column by column, starting from left to right.  
This method ensures that we understand the nature and quality of each field individually before applying any transformations. 

## Let's begin by examining the `Transaction_id` column

In [306]:
df['Transaction_id'].dtypes

dtype('O')

In [307]:
df['Transaction_id'].isna().sum()

0

In [308]:
df['Transaction_id'].duplicated().sum()

0

## Moving on to the next column: `Item`


In [309]:
df['Item'].dtypes

dtype('O')

In [310]:
df['Item'].value_counts()

Item
Juice       1171
Coffee      1165
Salad       1148
Cake        1139
Sandwich    1131
Smoothie    1096
Cookie      1092
Tea         1089
UNKNOWN      344
ERROR        292
Name: count, dtype: int64

In [311]:
df['Item'].isna().sum()

333

### Fixing Invalid `Item` Values Based on `Price_per_unit`

Some rows contain invalid or missing values in the `Item` column (e.g., "UNKNOWN", "ERROR", or NaN).  
We replaced them based on their `Price_per_unit` value only if the price clearly matches a single known item.  
Prices associated with multiple items (e.g., 4.0 → Smoothie/Sandwich) were not imputed to avoid incorrect assumptions.


In [312]:
df.groupby('Item').head(1) # to see each item  price


,Transaction_id,Item,Quantity,Price_per_unit,Total_spent,Payment_method,Location,Transaction_date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
5,TXN_2602893,Smoothie,5,4.0,20.0,Credit Card,NaN,2023-03-31
6,TXN_4433211,UNKNOWN,3,3.0,9.0,ERROR,Takeaway,2023-10-06
7,TXN_6699534,Sandwich,4,4.0,16.0,Cash,UNKNOWN,2023-10-28
14,TXN_8915701,ERROR,2,1.5,3.0,NaN,In-store,2023-03-21
17,TXN_6769710,Juice,2,3.0,6.0,Cash,In-store,2023-02-24
42,TXN_6650263,Tea,2,1.5,UNKNOWN,NaN,Takeaway,2023-01-10


In [313]:
df.groupby('Item')['Price_per_unit'].unique() 


Item
Cake                               [3.0, nan, UNKNOWN, ERROR]
Coffee                             [2.0, nan, ERROR, UNKNOWN]
Cookie                             [1.0, UNKNOWN, nan, ERROR]
ERROR       [1.5, 3.0, 5.0, nan, 4.0, 2.0, 1.0, UNKNOWN, E...
Juice                              [3.0, nan, UNKNOWN, ERROR]
Salad                              [5.0, ERROR, UNKNOWN, nan]
Sandwich                           [4.0, nan, ERROR, UNKNOWN]
Smoothie                           [4.0, nan, UNKNOWN, ERROR]
Tea                                [1.5, nan, ERROR, UNKNOWN]
UNKNOWN     [3.0, 1.0, 5.0, 4.0, 1.5, 2.0, nan, UNKNOWN, E...
Name: Price_per_unit, dtype: object

###  Fixing Incorrect or Inconsistent Prices

Some items are associated with multiple inconsistent prices, including invalid entries like 'ERROR' and 'UNKNOWN'.

To ensure consistency, we manually mapped each known item to its correct price and updated the `Price_per_unit` column accordingly.


In [314]:
fixed_prices = {
    'Coffee': 2.0,
    'Cake': 3.0,
    'Cookie': 1.0,
    'Salad': 5.0,
    'Smoothie': 4.0,
    'Sandwich': 4.0,
    'Juice': 3.0,
    'Tea': 1.5
}
def correct_price(row):
    item = row['Item']
    if item in fixed_prices:
        return fixed_prices[item]
    else:
        return row['Price_per_unit']
df['Price_per_unit'] = df.apply(correct_price, axis=1)


In [315]:
df.groupby('Item')['Price_per_unit'].unique()


Item
Cake                                                    [3.0]
Coffee                                                  [2.0]
Cookie                                                  [1.0]
ERROR       [1.5, 3.0, 5.0, nan, 4.0, 2.0, 1.0, UNKNOWN, E...
Juice                                                   [3.0]
Salad                                                   [5.0]
Sandwich                                                [4.0]
Smoothie                                                [4.0]
Tea                                                     [1.5]
UNKNOWN     [3.0, 1.0, 5.0, 4.0, 1.5, 2.0, nan, UNKNOWN, E...
Name: Price_per_unit, dtype: object

In [316]:
invalid_items = ['UNKNOWN', 'ERROR']
df['Item'] = df['Item'].replace(invalid_items, np.nan)

price_to_items = {
    2.0: 'Coffee',
    3.0: None,       
    1.0: 'Cookie',
    5.0: 'Salad',
    4.0: None,       
    1.5: 'Tea'
}
def infer_item(price):
    try:
        price = float(price)
        item = price_to_items.get(price)
        return item
    except:
        return None
df['Item'] = df.apply(
    lambda row: infer_item(row['Price_per_unit']) if pd.isna(row['Item']) else row['Item'],
    axis=1
)


In [317]:
df['Item'].value_counts()

Item
Coffee      1284
Salad       1270
Cookie      1209
Tea         1199
Juice       1171
Cake        1139
Sandwich    1131
Smoothie    1096
Name: count, dtype: int64

In [318]:
df['Item'].isna().sum()

501

At first glance, it may seem that the number of missing values has increased.  
However, this is because we replaced placeholder values such as `"UNKNOWN"` and `"ERROR"` with proper `NaN` values.  
Originally, the combined total of `"UNKNOWN"`, `"ERROR"`, and `NaN` entries was **969**, and after standardizing, the missing values are now correctly represented as **501 NaNs**.


In [319]:
df['Item'] = df['Item'].astype('category')

## Moving on to the next column: `Quantity` , `Price_per_unit` and `Total_spent`


In [320]:
df['Quantity'].dtypes

dtype('O')

In [321]:
df['Quantity'].value_counts()

Quantity
5          2013
2          1974
4          1863
3          1849
1          1822
UNKNOWN     171
ERROR       170
Name: count, dtype: int64

In [322]:
df['Quantity'].isna().sum()

138

### Fixing Invalid or Inconsistent `Quantity` Values

Some entries in the `Quantity` column may be missing or inconsistent with the corresponding `Total_spent` and `Price_per_unit` values.  
To correct this, we recalculated `Quantity` using the formula `Total_spent / Price_per_unit` .


In [323]:
df['Quantity'] = df['Quantity'].replace(['UNKNOWN', 'ERROR'], np.nan)
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce').astype('Int64')

df['Total_spent'] = df['Total_spent'].replace(['UNKNOWN', 'ERROR'], np.nan)
df['Total_spent'] = pd.to_numeric(df['Total_spent'], errors='coerce').astype('Float64')

df['Price_per_unit'] = pd.to_numeric(df['Price_per_unit'], errors='coerce').astype('Float64')


In [324]:
print(df['Total_spent'].isna().sum())
print(df['Price_per_unit'].isna().sum())

502
54


In [325]:
df['Total_spent'] = (df['Quantity'] * df['Price_per_unit']).where(
    lambda x: ~x.isna(), df['Total_spent']
)

df['Price_per_unit'] = (df['Total_spent'] / df['Quantity']).where(
    lambda x: ~x.isna(), df['Price_per_unit']
)

In [326]:
print(df['Total_spent'].isna().sum())
print(df['Price_per_unit'].isna().sum())

23
6


In [327]:
df['Quantity'].isna().sum()

479

In [328]:
df['Quantity'] = (df['Total_spent'] / df['Price_per_unit']).where(
    lambda x: ~x.isna(), df['Quantity']
)

In [329]:
df['Quantity'].isna().sum()

23

## Moving on to the next column: `Payment_method`


In [330]:
df['Payment_method'].dtypes

dtype('O')

In [331]:
df['Payment_method'].value_counts()

Payment_method
Digital Wallet    2291
Credit Card       2273
Cash              2258
ERROR              306
UNKNOWN            293
Name: count, dtype: int64

In [332]:
df['Payment_method'].isna().sum()

2579

In [333]:
df['Payment_method'] = df['Payment_method'].replace(['ERROR', 'UNKNOWN'], pd.NA)


### Handling Missing Values in `Payment_method`

Although the three valid payment methods (`Digital Wallet`, `Credit Card`, and `Cash`) are nearly equally distributed, the proportion of missing values is relatively large.  
Given this balance and to avoid introducing potential bias, we decided **not** to impute the missing values using the mode.  
Instead, we retained them as `NaN`, preserving the integrity of the original dataset.


In [334]:
df['Payment_method'] = df['Payment_method'].astype('category')

## Moving on to the next column: `Location`


In [335]:
df['Location'].dtypes

dtype('O')

In [336]:
df['Location'].value_counts()

Location
Takeaway    3022
In-store    3017
ERROR        358
UNKNOWN      338
Name: count, dtype: int64

In [337]:
df['Location'].isna().sum()

3265

In [338]:
df['Location'] = df['Location'].replace(['ERROR' , 'UNKNOWN'], pd.NA)

### Handling Missing Values in `Location`

The `Location` column contains two valid categories (`Takeaway` and `In-store`) with an almost even distribution.  
Due to this balance and the relatively high number of missing entries, we chose **not** to impute the missing values using the mode.  
Instead, we left them as `NaN` to maintain the neutrality of the dataset and avoid introducing any unintended bias.

In [339]:
df['Location'] = df['Location'].astype('category')

## Moving on to the next column: `Transaction_date`


In [340]:
df['Transaction_date'].dtypes

dtype('O')

This column contains dates, but they are not properly formatted and are currently being read as `object` instead of `datetime`, which prevents proper analysis and manipulation.  
At first glance, the dates appear to be separated by `-`, but we'll first check if there are any other common separators before proceeding with the conversion.


In [341]:
print(df['Transaction_date'].str.contains('-').sum())
print(df['Transaction_date'].str.contains('/').sum())
print(df['Transaction_date'].str.contains(' ').sum())

9540
0
0


In [342]:
df['Transaction_date'] = df['Transaction_date'].replace(['ERROR' , 'UNKNOWN'], pd.NA)
df['Transaction_date'].isna().sum()

460

In [343]:
df['Transaction_date'] = pd.to_datetime(df['Transaction_date'], errors='coerce')


In [344]:
df['Transaction_date'].isna().sum()

460

In [345]:
df.duplicated().sum()

0

In [346]:
df.isna().sum()

Transaction_id         0
Item                 501
Quantity              23
Price_per_unit         6
Total_spent           23
Payment_method      3178
Location            3961
Transaction_date     460
dtype: int64

In [347]:
df.dropna(subset=['Item','Quantity' , 'Price_per_unit' , 'Total_spent'] , inplace= True)

In [348]:
df.isna().sum()

Transaction_id         0
Item                   0
Quantity               0
Price_per_unit         0
Total_spent            0
Payment_method      3015
Location            3753
Transaction_date     435
dtype: int64